In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install pytorch_tabnet
!pip install cartopy
!pip install netCDF4

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 898.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import torch
from scipy.stats import gaussian_kde
from pytorch_tabnet.tab_model import TabNetRegressor
from matplotlib.colors import Normalize
import csv
import xarray as xr
import cartopy.crs as ccrs
from netCDF4 import Dataset
import plotly.graph_objects as go
from joblib import dump, load
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import SGDRegressor
import statsmodels.api as sm
import plotly.express as px
import plotly.subplots as sp
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import plotly.io as pio
pio.renderers.default = 'colab'

In [3]:
def nc_to_csv(folder_path):
    files = os.listdir(folder_path)
    files.sort()

    # Create the output directory if it doesn't exist
    # os.makedirs(output_path, exist_ok=True)

    for file_name in files:
        path = os.path.join(folder_path, file_name)
        reader = xr.open_dataset(path)
        reader_df = reader.to_dataframe()

        # Create a subdirectory for each file's CSV if it doesn't exist
        file_output_path = os.path.join(folder_path, os.path.splitext(file_name)[0])

        # Save CSV file
        reader_df.to_csv(file_output_path + '.csv')

In [4]:
# # Path to the folder in your Google Drive
# input = '/content/drive/My Drive/Colab Notebooks/global_data_0.25deg/Training-set'
# nc_to_csv(input)

In [6]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/global_data_0.25deg/Ibrahim_Calcite-data.csv')
df = df.dropna()
df.describe()

In [ ]:
# # Create a map
# plt.figure(figsize=(10, 6))
# ax = plt.axes(projection=ccrs.PlateCarree())
# ax.stock_img()

# # Plot TA values on the map
# sc = ax.scatter(val_df['lon'], val_df['lat'], c=val_df['ca'], cmap='viridis', transform=ccrs.PlateCarree())

# # Add colorbar
# plt.colorbar(sc, label='Calcium Carbonate values')

# # Add title and labels
# plt.title('Calcium Carbonate values on Map')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')

# # Show the plot
# plt.show()

In [ ]:
# da = df[['ca','sss','sst','chla']]
# target_variable = 'ca'
# correlation_with_target = da.corr()[target_variable].drop(target_variable)
# correlation_with_target.plot(kind='bar', color='black', width=0.3, figsize=(8, 6))
# plt.title(f'Correlation with {target_variable}', fontsize=16)
# plt.xticks(fontsize=14, rotation=45, ha='right')  # Adjust the fontsize as needed
# plt.xlabel('Variables', fontsize=14)
# plt.ylabel('Correlation', fontsize=14)
# plt.show()


In [ ]:
# fig = px.density_mapbox(
#     df,
#     lat='latitude',
#     lon='longitude',
#     z='ca',
#     zoom=2.35,
#     center = dict(lat=10.299, lon=87.062),
#     mapbox_style="stamen-terrain",
#     radius=8.5,
#     opacity=0.85,
#     title='Global TA Value'
# )

# fig.update_layout(mapbox_layers=[{
#     'below': 'traces',
#     'sourcetype': 'raster',
#     'source': [
#         'https://basemaps.cartocdn.com/rastertiles/light_all/{z}/{x}/{y}.png'
#     ]
# }])

# fig.show()

In [ ]:
X = df[['sst','sss','chla']]
y = df['ca']

In [ ]:
X = df[['sst','sss','chla']]
y = df['ca']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print("Minimum Value:",y_train.min())
print("Maximum Value:",y_train.max())
print("Mean Value:",y_train.mean())
print("Standard deviation:",y_train.std())

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
y_train = y.values.reshape(-1, 1)
# dump(scaler, '/content/drive/My Drive/Colab Notebooks/Models/ca_scaler')

In [ ]:
input_dim = 5
output_dim = 1
num_decision_steps = 5
num_features = 128
dropout_rate = 0.2
learning_rate = 0.001

tabnet_model = TabNetRegressor(
    n_d=num_features,
    n_a=num_features,
    n_steps=num_decision_steps,
    gamma=1.3,
    optimizer_params=dict(lr=learning_rate),
    scheduler_params={"step_size": 10, "gamma": 0.5},
    #mask_type="entmax",
    device_name="cuda" if torch.cuda.is_available() else "cpu"
)

tabnet_model.fit(
    X_train=X_train,
    y_train=y_train,
    max_epochs=100,
    patience=10,
    batch_size=1024,
    virtual_batch_size=1024,
    num_workers=0,
    drop_last=False,
)

tabnet_model.save_model('/content/drive/My Drive/Models/CA_tb_model')

In [ ]:
optimized_rf_model = RandomForestRegressor(n_estimators=75, random_state=42)
optimized_rf_model.fit(X_train, y_train.ravel())
dump(optimized_rf_model, '/content/drive/My Drive/Colab Notebooks/ca_model')

In [ ]:
tabnet_model = load('/content/drive/My Drive/Colab Notebooks/Models/CA_tb_model')
random_forest_model = load('/content/drive/My Drive/Colab Notebooks/Models/Ca_model')

In [ ]:
rf_pred = random_forest_model.predict(X_train)
tn_pred = tabnet_model.predict(X_train)

In [8]:
def mean_bias_error(true_values, predicted_values):
    errors = predicted_values - true_values
    mbe = np.mean(errors)
    return mbe

def mean_normalized_bias(observed_values, predicted_values):
    if len(observed_values) != len(predicted_values):
        raise ValueError("Input lists must have the same length.")

    n = len(observed_values)
    normalized_biases = [(observed - predicted) / observed for observed, predicted in zip(observed_values, predicted_values) if observed != 0]
    mnb = sum(normalized_biases) / len(normalized_biases) if len(normalized_biases) > 0 else 0

    return mnb

In [7]:
model = ['Metrics','Tabnet Model','Random Forest Model']
metrics_names = ['Root Mean Square Error','Mean Absolute Error','R-squared']

tn_metrics = [
    mean_bias_error(tn_pred, y_train),
    mean_normalized_bias(tn_pred, y_train),
    np.sqrt(mean_squared_error(tn_pred, y_train)),
    mean_absolute_error(tn_pred, y_train),
    r2_score(tn_pred, y_train)
]

rf_metrics = [
    mean_bias_error(rf_pred, y_train),
    mean_normalized_bias(rf_pred, y_train),
    np.sqrt(mean_squared_error(rf_pred, y_train)),
    mean_absolute_error(rf_pred, y_train),
    r2_score(rf_pred, y_train)
]

data = {'Metrics': metrics_names, 'Tabnet Model': rf_metrics, 'Random Forest Model': tn_metrics}
performance = pd.DataFrame(data, columns=model)
performance